# FIT5196 Assessment 1
<a id="FIT5196"></a>

#### Student Name: Jiawei Su
#### Student ID: 29590183


#### Student Name: Weiwei Jin
#### Student ID: 28106946

Date: 25/08/2019

Version: 1.0

Environment: Python 3.7.1 and Jupyter notebook

Libraries used: 
* pandas (for dataframe, included in Anaconda Python 3.7.1) 
* re (for regular expression, included in Anaconda Python 3.7.1) 


# Table of contents

* [Student Information](#FIT5196)
* [1. Introduction](#Introduction)
* [2. Import libraries](#Lib)
* [3. Regular Expressions Design](#Reg)
   * [3.1. Approach](#app)
   * [3.2. Patterns](#pat)
   * [3.2. Explanation of Patterns](#Expat)
* [4. Functions](#Functions)
* [5. Processing the file](#Processing)
   * [5.1. Read file](#read)
   * [5.2. Process data and generate CSV file](#csv)
   * [5.3. Process data and generate JSON file](#json)
* [6. Summary](#Summary)

## 1.  Introduction
<a id="Introduction"></a>

In this task, a text file `Group067.txt` will be provided, all the data in the text file should be processed. Follow the assignment 1 requirement, the data in the text should be parsed and certain elements of data should be extracted. After the data is wrangled, it will be converted to `Group067.csv` file and `Group067.json` in the provided format.

## 2.  Import libraries
<a id="Lib"></a>

In [1]:
# import libraries used in this assessment

# Pandas library provides high-performance, easy-to-use data structures 
# and data analysis tools
import pandas as pd

# Re library provides regex related functions
import re

# CSV library is used to process CSV file
import csv

# JSON library is used to process JSON file
import json

## 3. Regular Expressions Design
<a id="Reg"></a>

In this section, we have defined the Regular expression patterns we used to extract data from text file.

### 3.1 Approach
<a id="app"></a>
The approach we designed to extract data elements from the provided `.txt` file is stated as below:

<div class="alert alert-block alert-info">
    
1. Read `.txt` file, separate the file into blocks of patents by using the line `<?xml version="1.0" encoding="UTF-8"?>` we have identified.


2. For `150` blocks of data, we identify patterns and use them to extract different elements which will be explained in `section 3.2`.


3. There are math formulas in the text. We have decided to remove them (both `regular math formula` and `in-line math formula`) as they cannot be shown correctly in `.csv` file or `.json` file. We will keep the variables in text.


4. We put the lists of different element result together using either `Pandas` or `Zip` based on assessment requirements.

### 3.2 Patterns
<a id="pat"></a>
The regular expression patterns and other useful constants we use in this assignment is below:

In [2]:
FILE_PATH = 'Group067.txt'

# Define the output path for csv document
CSV_OUTPUT_PATH = 'Group067.csv'      

# Define the output path for json document
JSON_OUTPUT_PATH = 'Group067.json'    

# Using regular expression patterns to extract trageted data
ID_PATTERN = re.compile('(<document-id>.+?</document-id>)')      
ID_CONTENT_PATTERN = re.compile('<publication-reference>(.*?)</publication-reference>')

KIND_CONTENT_PATTERN = re.compile('appl-type\s*=\s*"(.+?)"')

TITLE_CONTENT_PATTERN = re.compile('<invention-title id=".*">(.*?)</invention-title>')

CLAIM_CONTENT_PATTERN = re.compile('<number-of-claims>(.*?)</number-of-claims>')

EXAM_CITE_CONTENT_PATTERN = re.compile('<category>cited by examiner</category>')

APP_CITE_CONTENT_PATTERN = re.compile('<category>cited by applicant</category>')

INVENTORS_GROUP_PATTERN = re.compile('<inventors>(.*?)</inventors>')
INVENTORS_CONTENT_PATTERN = re.compile('<inventor sequence="[^"]*" designation="[^"]*">(.*?)</inventor>')
FIRST_NAME_CONTENT_PATTERN = re.compile('<first-name>(.*?)</first-name>')
LAST_NAME_CONTENT_PATTERN = re.compile('<last-name>(.*?)</last-name>')

CLAIMS_CONTENT_PATTERN = re.compile('<claim id="[^"]*" num="[^"]*">(.*?)</claim>')
TAG_REMOVE_PATTERN = re.compile('</?(.+?)>')
MATH_REMOVE_PATTERN = re.compile('<maths id="[^"]*" num="[^"]*">(.*?)</maths>')
FORMULA_REMOVE_PATTERN = re.compile('<[?]in-line-formulae description="In-line Formulae" end="lead"[?]>(.*?)<[?]in-line-formulae description="In-line Formulae" end="tail"[?]>')

ABSTRACT_CONTENT_PATTERN = re.compile('<abstract id="abstract">(.*?)</abstract>')

# A list consists of the name of required data elements
NAME_ROW = ["grant_id", "patent_title", "kind", "number_of_claims", "inventors", 
            "citations_applicant_count", "citations_examiner_count", "claims_text", "abstract"]

### 3.3 Explanation of Patterns
<a id="Expat"></a>
Explanation of our designed regular expression and the approach we design them is below:

<div class="alert alert-block alert-warning">
    
1. **`ID_PATTERN`** and **`ID_CONTENT_PATTERN`** is used to extract: country code(which sits between `<country>` and `</country>`) and id number(which sits between `<doc-number>` and `</doc-number>`) and concatenate them to get ID. Full approach in function **grant_id**.


2. **`KIND_CONTENT_PATTERN`** is used to extract: patent kind(which sits in `<application-reference >` as the value of the attribute `appl-type`). Full approach in function **kind**.


3. **`TITLE_CONTENT_PATTERN`** is used to extract: patent title(which sits between `<invention-title>` and `</invention-title>`). Full approach in function **patent_title**.


4. **`CLAIM_CONTENT_PATTERN`** is used to extract: the number of claims(which sits between `<number-of-claims>` and `</number-of-claims>`). Full approach in function **number_of_claims**.


5. **`EXAM_CITE_CONTENT_PATTERN`** is used to extract: the number of examiner citations(where the text between `<category>` and `</category>` in each block is `cited by examiner`), we count the number of this text we scanned frm the text. Full approach in function **citations_examiner_count**.


6. **`APP_CITE_CONTENT_PATTERN`** is used to extract: the number of applicant citations(where the text between `<category>` and `</category>` in each block is `cited by applicant`), we count the number of this text we scanned frm the text. Full approach in function **citations_applicant_count**.


7. **`INVENTORS_GROUP_PATTERN`** and **`INVENTORS_CONTENT_PATTERN`** is used to extract: all the authors(which sits between `<inventors>` and `</inventors>`) for each block and from which we can separate each author's detail(which sits between `<inventor >` and `</inventor>`). Then we use **`FIRST_NAME_CONTENT_PATTERN`** and **`LAST_NAME_CONTENT_PATTERN`** to  extract: First Name(which sits between `<first-name>` and `</first-name>`) and Last Name(which sits between `<last-name>` and `</last-name>`) and concatenate them to get the full name. Full approach in function **inventors**.


8. **`CLAIMS_CONTENT_PATTERN`** is used to extract: claim content(which sits between `<claim >` and `</claim>`). Then we first need to remove Math formula using **`MATH_REMOVE_PATTERN`** and **`FORMULA_REMOVE_PATTERN`** ,then we need to remove all tags using **`TAG_REMOVE_PATTERN`**. Lastly we concatenate all the claims. Full approach in function **claims_text**.


9. **`ABSTRACT_CONTENT_PATTERN`** is used to extract: abstract(which sits between `<abstract >` and `</abstract>`). Then we first need to remove Math formula using **`MATH_REMOVE_PATTERN`** and **`FORMULA_REMOVE_PATTERN`** ,then we need to remove all tags using **`TAG_REMOVE_PATTERN`**. Full approach in function **abstract**.

<div class="alert alert-block alert-warning">

1. **`TAG_REMOVE_PATTERN`**: Remove useless tags and text between `<` and `>`.


2. **`MATH_REMOVE_PATTERN`**: Used to clean maths equations between `<maths >` and `</maths>` which we don't need to handle in this assignment.


3. **`FORMULA_REMOVE_PATTERN`**: Used to clean in line math formulas between `<?in-line-formulae description="In-line Formulae" end="lead"?>` and `<?in-line-formulae description="In-line Formulae" end="tail"?>` which we don't need to handle in this assignment.

### 4. Functions
<a id="Functions"></a>

In this secition, we define a couple of functions prior to processing the file. The goal is to make it simpler when we use the them to extract the data later on.

In [3]:
# Function to read the a text file, preprocess it, add the content of the text file to a list and return the list.
def read_file(path):        
    content = []                                         
    try:
        with open(path, 'r', encoding = 'utf-8') as fp: 
            for line in fp:
                if '<?xml version="1.0" encoding="UTF-8"?>' in line:
                    content.append('')
                content[-1] += line.strip('\n')
    finally:
        fp.close()
    return content

# Function to find the required data(patent id) and return the results in a list
def grant_id(content):                                   
    result = []

    # Use ID_CONTENT_PATTERN to findout the country and document number and put them together as the id
    for entry in content:    
        block = ID_CONTENT_PATTERN.findall(entry)       
        result.append(re.findall(r'(<country>(.*?)</country>)', block[0])[0][1] 
                      + re.findall(r'(<doc-number>(.*?)</doc-number>)', block[0])[0][1])
    return result

# Function to find the required data(patent id), pre-process it and return the result in a list
def kind(content):
    result = []
    for entry in content:
        raw_opt = KIND_CONTENT_PATTERN.findall(entry)
        if "utility" in raw_opt:
            result.append("Utility Patent Grant (with a published application) issued on or after January 2, 2001.")
        else:
            result.append(raw_opt[0].capitalize() + " Patent")
    return result

# Function to find the required data(patent title) and return the result in a list
def patent_title(content):  
    result = []
    for entry in content:
        result.append(re.findall(TITLE_CONTENT_PATTERN, entry)[0])
    return result

# Function to find the required data(number of claims) and return the result in a list
def number_of_claims(content): 
    result = []
    for entry in content:
        result.append(re.findall(CLAIM_CONTENT_PATTERN, entry)[0])
    return result

# Function to find the required data(ciatations examiner count) and return the result in a list
def citations_examiner_count(content):
    result = []
    for entry in content:
        result.append(len(re.findall(EXAM_CITE_CONTENT_PATTERN, entry)))
    return result

# Function to find the required data(application citation content) and return the result in a list
def citations_applicant_count(content): 
    result = []
    for entry in content:
        result.append(len(re.findall(APP_CITE_CONTENT_PATTERN, entry)))
    return result

# Function to find the required data(inventors' name) and return the result in a list
def inventors(content):     
    result = []
    for entry in content:
        stri = "["
        author = []
        inv_block = re.findall(INVENTORS_GROUP_PATTERN, entry)[0]  
        for inv in re.findall(INVENTORS_CONTENT_PATTERN, inv_block):
            author.append(re.findall(FIRST_NAME_CONTENT_PATTERN, inv)[0] 
                          + " " + re.findall(LAST_NAME_CONTENT_PATTERN, inv)[0])
        # If the list is not empty, add the list of authors in the result list or if it is empty, add "NA"
        if author:                      
            for loc, au in enumerate(author):
                stri += au
                if loc < len(author) - 1:
                    stri += ","
                if loc == len(author) - 1:
                    stri += "]"
            result.append(stri)
        else:
            result.append("NA")
    return result

# Function to find the required data(claims text) and return the result in a list 
def claims_text(content):           
    result = []
    for entry in content:           
        block = re.findall(CLAIMS_CONTENT_PATTERN, entry)
        this_claim = []
        stri = "["
         
        # Add the result into the result list
        for claim in block:
            # Clean Math formulas
            new_claim = re.sub(MATH_REMOVE_PATTERN, '', claim)
            
            # Clean in-line math formulas
            a_new_claim = re.sub(FORMULA_REMOVE_PATTERN, '', new_claim)
            
            # Replece all the tags(and contents between "</>") in the claims_content_pattern with ''
            this_claim.append(re.sub(TAG_REMOVE_PATTERN, '', a_new_claim))
        # If the required data is found, add to the result list, if not just append "NA"
        if this_claim:            
            for loc, cl in enumerate(this_claim):
                stri += cl
                if loc < len(this_claim) - 1:
                    stri += ","
                if loc == len(this_claim) - 1:
                    stri += "]"
            result.append(stri)
        else:
            result.append("NA")
    return result

# Function to find the required data(abstract) and return the result in a list 
def abstract(content):
    result = []
    # Add the data to the result list
    for entry in content:         
        result.append(re.findall(ABSTRACT_CONTENT_PATTERN, entry))
    for count in range(len(result)):
        # If not data can be found, append "NA"
        if not result[count]:
            result[count] = 'NA'
        # If there are matched result, replace the tag(and the contents between "</>") with ''.
        else:                         
            result[count] = re.sub(TAG_REMOVE_PATTERN, '', result[count][0])
    return result

## 5.  Processing the file
<a id="Processing"></a>

In this section, we process the file using the defined function, and then generate output files in the format of CSV using pandas and JSON using traditional method.

### 5.1 Read file
<a id="read"></a>
Read the file defined in the file path into a list called `src`

In [4]:
# Read the file defined in the file path into a list
src = read_file(FILE_PATH)

### 5.2 Process data and generate CSV file
<a id="csv"></a>
Process the data and then output into a CSV file

In [5]:
# A list of functions which will be applied to the list of data
parsing_alrogithms = [grant_id, patent_title, kind, number_of_claims, inventors,    
                      citations_applicant_count, citations_examiner_count, claims_text, abstract]

# Define an empty dictionay
dic = {}

# Apply functions in list to data list and then add the result to the dictionary under the key of its function name
for funct in parsing_alrogithms:        
    dic[funct.__name__] = funct(src)

# Using pandas's dataframe to store the dictionary with using the defined table's column name 
df = pd.DataFrame(dic, columns = NAME_ROW)

# Output the dataframe to a CSV file
df.to_csv(CSV_OUTPUT_PATH, index = None, header=True)

### 5.3 Process data and generate JSON file
<a id="json"></a>
Process the data and then output into a JSON file

In [6]:
# Apply functions to extract data and store into separate lists
list_id = grant_id(src)
list_kind = kind(src)
list_title = patent_title(src)
list_no_claims = number_of_claims(src)
list_exam_cite = citations_examiner_count(src)
list_app_cite = citations_applicant_count(src)
list_inv = inventors(src)
list_claim = claims_text(src)
list_abst = abstract(src)

# Zip the lists together(without the list of IDs)
zipped = zip(list_kind, list_title, list_no_claims, list_exam_cite,  
            list_app_cite, list_inv, list_claim, list_abst)

info = []
for el in zipped:
    # Process each entry of the zipped result and add data into the new dictionary
    dict_temp = {}
    dict_temp["patent_title"] = el[1]
    dict_temp["kind"] = el[0]
    dict_temp["number_of_claims"] = int(el[2])
    dict_temp["inventors"] = el[5]
    dict_temp["citations_applicant_count"] = el[4]
    dict_temp["citations_examiner_count"] = el[3]
    dict_temp["claims_text"] = el[6]
    dict_temp["abstract"] = el[7]
    
    # Add the dictionary of contents into a list
    info.append(dict_temp)

# Zip the ids with its information and then add them into a new dictionary
j_dict = {}              
for each in zip(list_id, info):
    j_dict[each[0]] = each[1]

# Dump the dictionary into a string
json_str = json.dumps(j_dict)  
with open(JSON_OUTPUT_PATH, 'w') as json_file:
    # Write the json string to a new json file
    json_file.write(json_str)

## 6. Summary
<a id="Summary"></a>

In this assignment, we are required to parse a text document and output into a JSON file and CSV document. Firstly, we use the regular expression to scan all the required elements and save them into result lists. Secondly, we use a pandas DatatFrame structure to output the contents into a CSV file. Lastly, we use use the traditional method to zip the results of different elements and then output the result into a JSON file. Takeaways of this assessmnet is:

- **Applying Regular Expression** : We explore differnet techniques of regular expression to help us extract useful data element from text. Useful functions in `re` library are `findall()` and `sub()`.


- **Pandas Dataframe** : We use Pandasdataframe to store data, this is an easier and simpler format. `to_csv()` function can be used to generate an `.csv` outout of a Dataframe .


- **Combining Lists** : The `zip()` function returns a zip object, which is an iterator of tuples where the first item in each passed iterator is paired together, and then the second item in each passed iterator are paired together, etc. 


- **List Traversal** : The `enumerate()` function helps us an easier way to traverse a list. 